# Simulation study

In [ ]:
import pandas as pd
import numpy as np
from pysurvival import utils
from pysurvival.models import BaseModel
from random import choices

## Data Simulation

In [ ]:
class SimulationModel(BaseModel):
    def __init__(self, censored_parameter = 2, alpha = 1, beta = 1, bins = 100, risk_parameter = 1, survival_distribution = 'exponential'):
        # Saving the attributes
        self.censored_parameter = censored_parameter
        self.alpha              = alpha 
        self.beta               = beta
        self.risk_parameter     = risk_parameter
        self.bins               = bins
        self.features           = []
        self.survival_distribution = survival_distribution
        self.risk_type = 'linear'

        # Initializing the elements from BaseModel
        super(SimulationModel, self).__init__(auto_scaler = True)


    @staticmethod
    def generate_column_data(N, column):
        distributions = {
            'sex': np.random.binomial(n = 1, p = 0.5, size = N),
            'age': np.random.randint(low = 55, high = 85, size = N),
            'education': np.random.randint(low = 0, high = 36, size = N),
            'memory': choices(population=[0, 1], weights=[0.917, 0.083], k=N),
            'judgment': choices(population=[0, 1], weights=[0.96, 0.04], k=N),
            'decclin': choices(population=[0, 1], weights=[0.95, 0.05], k=N),
            'travel_1': choices(population=[0, 1], weights=[0.97, 0.03], k=N),
            'travel_2': choices(population=[0, 1], weights=[0.98, 0.02], k=N),
            'motrem': choices(population=[0, 1], weights=[0.99, 0.01], k=N),
            'cogstat_1': choices(population=[0, 1], weights=[0.84, 0.16], k=N),
            'cogstat_9': choices(population=[0, 1], weights=[0.86, 0.14], k=N),
            'decin': choices(population=[0, 1], weights=[0.88, 0.12], k=N)
            }

        return distributions[column]

  
    def time_function(self, BX):
        # Calculating scale coefficient using the features
        num_samples = BX.shape[0]
        lambda_exp_BX = np.exp(BX)*self.alpha
        lambda_exp_BX = lambda_exp_BX.flatten()

        # Generating random uniform variables
        U = np.random.uniform(0, 1, num_samples)

        # Exponential
        if self.survival_distribution == 'exponential':
            self.survival_distribution = 'Exponential'
            return - np.log( U )/( lambda_exp_BX )
      
        # Weibull 
        elif self.survival_distribution == 'weibull':
            self.survival_distribution = 'Weibull'
            return np.power( - np.log( U )/( lambda_exp_BX ), 1./self.beta )

        # Log-Logistic 
        elif self.survival_distribution == 'log-logistic':
            self.survival_distribution = 'Log-Logistic'
            return  np.power( U/(1.-U), 1./self.beta )/(lambda_exp_BX )


    def hazard_function(self, t, BX):
        # Calculating scale coefficient using the features
        _lambda = self.alpha*np.exp( BX )

        # Exponential
        if self.survival_distribution == 'exponential':
            return  np.repeat(_lambda, len(t))
        
        # Weibull 
        elif self.survival_distribution == 'weibull':
            return  _lambda*self.beta*np.power( t, self.beta-1 )
        
        # Log-Logistic 
        elif self.survival_distribution == 'log-logistic':
            numerator =  _lambda*self.beta*np.power((_lambda*t), self.beta-1 )
            denominator =  (1 + np.power( (_lambda*t), self.beta) )
            return numerator/denominator


    def survival_function(self, t, BX):
        # Calculating scale coefficient using the features
        _lambda = self.alpha*np.exp( BX )

        # Exponential
        if self.survival_distribution == 'exponential':
            return np.exp( -t*_lambda )
        
        # Weibull 
        elif self.survival_distribution == 'weibull':
            return np.exp( -np.power(t, self.beta)*_lambda )
        
        # Log-Logistic 
        elif self.survival_distribution == 'log-logistic':
            return 1./(1.+ np.power(_lambda*t, self.beta) )


    def risk_function(self, x_std, gender=''):
        # type of risk - linear only
        if self.heterogeneity == False:
            risk = np.dot( x_std, self.feature_weights )
            return risk.reshape(-1, 1)
    
        else:
            # choose gender
            if (gender == 'men'):
                risk = np.dot( x_std, self.men_feature_weights)
                return risk.reshape(-1, 1)

            if (gender == 'women'):
                risk = np.dot( x_std, self.women_feature_weights)
                return risk.reshape(-1, 1)


    def generate_data(self, num_samples = 100, feature_weights = [], men_feature_weights = [], women_feature_weights = [], columns = [], heterogeneity = False):
        # Data parameters
        self.num_variables = len(columns)
        self.heterogeneity = heterogeneity

        if heterogeneity == False:
            feature_weights = utils.check_data(feature_weights)
            if self.num_variables != len(feature_weights):
                error = "The length of feature_weights ({}) "
                error += "and num_variables ({}) are not the same."
                error = error.format(len(feature_weights), self.num_variables)
                raise ValueError(error)
            self.feature_weights = feature_weights

        else:
            # weight for men
            men_feature_weights = utils.check_data(men_feature_weights)
            if self.num_variables != len(men_feature_weights):
                error = "The length of men_feature_weights ({}) "
                error += "and num_variables ({}) are not the same."
                error = error.format(len(men_feature_weights), self.num_variables)
                raise ValueError(error)
            self.men_feature_weights = men_feature_weights

            # weight for women
            women_feature_weights = utils.check_data(women_feature_weights)
            if self.num_variables != len(women_feature_weights):
                error = "The length of feature_weights ({}) "
                error += "and num_variables ({}) are not the same."
                error = error.format(len(women_feature_weights), self.num_variables)
                raise ValueError(error)
            self.women_feature_weights = women_feature_weights


        # Generating and creating features
        X = np.zeros((num_samples, self.num_variables))

        for index, col_name in enumerate(columns):
            X[:, index] = self.generate_column_data(num_samples, col_name)

        if heterogeneity == False:
            X_std = self.scaler.fit_transform( X )
            BX = self.risk_function( X_std )
        # if heterogeneity is True
        else:
            # sort -> first men, than women
            X = X[X[:, 0].argsort()]
            # split data based on gender
            pom_df = pd.DataFrame(X)
            df_0, df_1 = [x for _, x in pom_df.groupby(pom_df.iloc[:, 0] == 1)]
            X_men = df_0.to_numpy()
            X_women = df_1.to_numpy()

            # men
            X_std_men = self.scaler.fit_transform( X_men )
            BX_men = self.risk_function( X_std_men, gender='men' )

            # women
            X_std_women = self.scaler.fit_transform( X_women )
            BX_women = self.risk_function( X_std_women, gender='women' )

            BX = np.concatenate((BX_men, BX_women))

        # Building the survival times
        T = self.time_function(BX) 
        C = np.random.normal( loc = self.censored_parameter, scale = 3.611, size = num_samples)
        C = np.maximum(C, 0.)
        N_zeros = C.size - np.count_nonzero(C)
        
        while(N_zeros != 0):
            C[np.where(C == 0)] = np.random.normal(loc = self.censored_parameter, scale = 3.611, size = 1)
            C = np.maximum(C, 0.)
            N_zeros = C.size - np.count_nonzero(C)
        
        time = np.minimum( T, C )
        E = 1.*(T == time)

        # Building dataset
        self.features = columns
        self.dataset = pd.DataFrame(data = np.c_[X, time, E], columns = columns + ['time', 'event'])

        # Building the time axis and time buckets
        self.times = np.linspace(0., max(self.dataset['time']), self.bins)
        self.get_time_buckets()

        # Building baseline functions
        self.baseline_hazard   =  self.hazard_function(self.times, 0)
        self.baseline_survival =  self.survival_function(self.times, 0) 

        # Printing summary message
        message_to_print = "Survival distribution: {}\nNumber of data-points: {} - Number of events: {}"
        print( message_to_print.format(self.survival_distribution, num_samples, sum(E)) )

        return self.dataset

## Generate data with given survival distribution

In [ ]:
# survival distribution: exponential, weibull or log-logistic
survival_dist = 'exponential'
# survival_dist = 'weibull'
# survival_dist = 'log-logistic'

N_simulations = 150
N_samples = 100

simulation_model = SimulationModel(survival_distribution=survival_dist)

# weights
w_sex = -0.185
w_age = 0.07
w_education = -0.016
w_memory = 0.287
w_judgment = 0.207
w_decclin = 0.313
w_travel_1 = 0.381
w_travel_2 = 0.418
w_motrem = -0.559
w_cogstat_1 = 0.570
w_cogstat_9 = -0.440
w_decin = 0.497

weights = [w_sex, w_age, w_education, w_memory, w_judgment, w_decclin, w_travel_1, w_travel_2, w_motrem, w_cogstat_1, w_cogstat_9, w_decin]
columns = ['sex', 'age', 'education', 'memory', 'judgment', 'decclin', 'travel_1', 'travel_2', 'motrem', 'cogstat_1', 'cogstat_9', 'decin']

dataset = simulation_model.generate_data(num_samples=N_samples, feature_weights=weights, columns=columns)
# dataset

## Generate data with given heterogeneity

**Small observed heterogeneity:**

male: $\beta_1 = 0.57$, $\beta_9 = −0.44$

female: $\beta_1 = 0.57 − 0.13$, $\beta_9 = −0.44 − 0.31$

**Medium observed heterogeneity:**

male: $\beta_1 = 0.57$, $\beta_9 = −0.44$

female: $\beta_1 = 0.57 − (0.13 + 0.13)$, $\beta_9 = −0.44 − (0.31 + 0.16)$

**Large observed heterogeneity:**

male: $\beta_1 = 0.57$, $\beta_9 = −0.44$

female: $\beta_1 = 0.57 − (0.13 + 2*0.13)$, $\beta_9 = −0.44 − (0.31 + 2*0.16)$

In [ ]:
survival_dist = 'exponential'
N_simulations = 150
N_samples = 100

simulation_model = SimulationModel(survival_distribution=survival_dist)

# weights
w_sex = -0.185
w_age = 0.07
w_education = -0.016
w_memory = 0.287
w_judgment = 0.207
w_decclin = 0.313
w_travel_1 = 0.381
w_travel_2 = 0.418
w_motrem = -0.559
# cogstat men
w_cogstat_1_men = 0.570
w_cogstat_9_men = -0.440
# cogstat womem - small heterogeneity
w_cogstat_1_women = 0.44
w_cogstat_9_women = -0.75
# # cogstat womem - medium heterogeneity
# w_cogstat_1_women = 0.31
# w_cogstat_9_women = -0.91
# # cogstat womem - large heterogeneity
# w_cogstat_1_women = 0.18
# w_cogstat_9_women = -1.07
w_decin = 0.497

weights_men = [w_sex, w_age, w_education, w_memory, w_judgment, w_decclin, w_travel_1, w_travel_2, w_motrem, w_cogstat_1_men, w_cogstat_9_men, w_decin]
weights_women = [w_sex, w_age, w_education, w_memory, w_judgment, w_decclin, w_travel_1, w_travel_2, w_motrem, w_cogstat_1_women, w_cogstat_9_women, w_decin]
columns = ['sex', 'age', 'education', 'memory', 'judgment', 'decclin', 'travel_1', 'travel_2', 'motrem', 'cogstat_1', 'cogstat_9', 'decin']

dataset = simulation_model.generate_data(num_samples=N_samples, men_feature_weights=weights_men, women_feature_weights=weights_women, columns=columns, heterogeneity=True)
# dataset